<a href="https://colab.research.google.com/github/Ashish265/hands_on_Large_Language_models/blob/main/Chapter_10_Creating_Text_Embedding_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install -q accelerate>=0.27.2 peft>=0.9.0 bitsandbytes>=0.43.0 transformers>=4.38.2 trl>=0.7.11 sentencepiece>=0.1.99
%pip install -q sentence-transformers>=3.0.0 mteb>=1.1.2 datasets>=2.10.0

Creating an embedding model

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("glue", "mnli",split="train")
train_dataset =  train_dataset.remove_columns("idx")

In [3]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

Model

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('bert-base-uncased')

Loss Function

In [6]:
from sentence_transformers import losses

train_loss = losses.SoftmaxLoss(
    model=embedding_model,
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(),
    num_labels=3
)

Evaluation

In [ ]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

val_sts = load_dataset(
    "glue",
    "stsb",
    split='validation'
)

evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts['sentence2'],
    scores=[score/5 for score in val_sts['label']],
    main_similarity='cosine'

)

Trainig

In [10]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

args  = SentenceTransformerTrainingArguments(
    output_dir='base_embedding_model',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100
)


In [16]:
from sentence_transformers.trainer import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=embedding_model,
    args=args,
    train_dataset=train_dataset,
    loss=train_loss,
    evaluator=evaluator
)

In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ashishgupta2692 (ashishgupta2692-samsung-electronics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,0.755500
200,0.961600
300,0.866800
400,0.841700
500,0.825800
600,0.824800
700,0.808400
800,0.798100
900,0.792500
1000,0.770300


In [ ]:
# Evaluate our trained model
evaluator(embedding_model)